In [0]:
!pip install sentence-transformers

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 450kB 7.1MB/s 
     |████████████████████████████████| 1.0MB 24.2MB/s 
     |████████████████████████████████| 870kB 32.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5.1-cp36-none-any.whl size=67076 sha256=99f9d9befdb38cc816301d02107e0a41086f9dec78ecc3eb451afbf9609822c5
  Stored in directory: /root/.cache/pip/wheels/22/ca/b4/7ca542b411730a8840f8e090df2ddacffa1c4dd9f209684c19
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=0a1b4c3cda1a1877b44a4661772619d569236678543efbae80c9d1fa4b3048f5
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sentence-transformers sacremoses


In [0]:
from sentence_transformers import SentenceTransformer
import scipy.spatial

embedder = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:18<00:00, 22.2MB/s]


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import re

usecols = (4,7,8)
train_dataframe = pd.read_table("/content/gdrive/My Drive/task3_training.tsv",usecols=usecols)
test_dataframe = pd.read_table("/content/gdrive/My Drive/task3_validation.tsv",usecols=usecols)

print(train_dataframe.shape)
print(test_dataframe.shape)
print(train_dataframe.head())

(2246, 3)
(560, 3)
  extraction  meddra_code meddra_term
0        NaN          NaN         NaN
1        NaN          NaN         NaN
2        NaN          NaN         NaN
3        NaN          NaN         NaN
4        NaN          NaN         NaN


In [0]:
train_term = train_dataframe.iloc[:, 2]
train_code = train_dataframe.iloc[:, 1]
train_data = train_dataframe.iloc[:, 0]

test_data = test_dataframe.iloc[:, 0]
test_code = test_dataframe.iloc[:, 1]
test_term = test_dataframe.iloc[:, 2]
print(train_code[0])
print(train_data[0])
print(type(train_code[0]))
print(type(train_code[30]))
#print(train_data)

nan
nan
<class 'numpy.float64'>
<class 'numpy.float64'>


In [0]:
extracted_train_term =[] #med term : med code
extracted_train_code =[]
extracted_train_data =[]
for index, data in enumerate(train_term):
  if type(data) == str:
    #print(index, data, train_data[index], train_code[index])
    extracted_train_term.append(data)
    extracted_train_code.append(train_code[index])
    extracted_train_data.append(train_data[index])


extracted_test_data = []
extracted_test_term = [] #med term : med code
extracted_test_code =[]
for index, data in enumerate(test_term):
  if type(data) == str:
    extracted_test_term.append(data)
    extracted_test_code.append(test_code[index])
    extracted_test_data.append(test_data[index])

In [0]:
corpus = extracted_train_term #+ extracted_test_term

print(corpus)

['vision loss', 'withdrawal syndrome', 'nauseous', 'inability to orgasm', 'weight gain', 'weight gain', 'headache', 'condition worsened', 'appetite suppressed', 'pain', 'drug toxicity', 'feeling unwell', 'confusion', 'dizziness', 'spaced out', 'increasd skin sensitivity', 'rash', 'feeling strange', 'sleeplessness', 'headache', 'hair loss', 'hair loss', 'sleeplessness', 'calcium deficiency', 'vitamin d deficiency', 'depression', 'weight gain', 'weight gain', 'liver damage', 'allergic reaction', 'near death experience', 'mixed manic depressive episode', 'feeling high', 'withdrawal syndrome', 'pain', 'pain in heel', 'pain', 'arm injury', 'migraine', 'vomiting', 'illogical thinking', 'slowed thinking', 'feeling irritated', 'felt like a zombie', 'somnolence', 'ache', 'flu like symptoms', 'shivers', 'sweating', 'rage', 'ache', 'back pain', 'fluid retention', 'swelling of legs', 'drug withdrawal syndrome', 'electric shock sensation', 'lack of feeling emotions', 'lack of feeling emotions', 'an

In [0]:
corpus_embeddings = embedder.encode(corpus)

In [0]:
queries = extracted_test_data
query_embeddings = embedder.encode(queries)

In [0]:
def find_distance(queries, query_embeddings, corpus, corpus_embeddings, \
                  closest_n, actual_adr, actual_adr_code, corpus_train_code):
  index = 0
  correct = 0
  file = open(r"/content/gdrive/My Drive/results_task3.txt","w") 
  for adr, query_embedding in zip(queries, query_embeddings):
      distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

      results = zip(range(len(distances)), distances)
      results = sorted(results, key=lambda x: x[1])

      print("\n\n======================\n\n")
      file.write("\n\n======================\n\n")
      print("Extracted ADR from tweet: ",adr)
      file.write("Extracted ADR from tweet: "+ adr)

      for idx, distance in results[0:closest_n]:
          print("\nMatched ADR term in corpus:", corpus[idx].strip(), '\nMatched ADR code', int(corpus_train_code[idx]),\
                "(Score: %.4f)" % (1-distance))
          file.write("\nMatched ADR term in corpus:"+ corpus[idx].strip()+ '\nMatched ADR code'+ str(int(corpus_train_code[idx]))+\
                "(Score: %.4f)" % (1-distance))
          print('\nExpected ADR term : ', actual_adr[index], '\nExpected ADR code : ', int(actual_adr_code[index]))
          file.write('\nExpected ADR term : '+ actual_adr[index]+ '\nExpected ADR code : '+ str(int(actual_adr_code[index])))
          if(actual_adr[index].strip() == corpus[idx].strip()):
            correct += 1
            print("\nResult: Correct")
            file.write("\nResult: Correct")
          else:
            print("\nResult: Wrong")
            file.write("\nResult: Wrong")
          index += 1
  print('\nTotal Extracted ADR Evaluated: ', len(actual_adr) ,' Correctly Identified: ', correct)
  print('Accuracy', (correct/float(len(actual_adr)))*100)
  file.write('\nTotal Extracted ADR Evaluated: '+ str(len(actual_adr)) +' Correctly Identified: '+ str(correct))
  file.write('Accuracy: '+ str((correct/float(len(actual_adr)))*100))
  file.close()

In [0]:
closest_n = 1
find_distance(queries, query_embeddings, corpus, corpus_embeddings, closest_n,\
              extracted_test_term, extracted_test_code, extracted_train_code)

Streaming output truncated to the last 5000 lines.


Extracted ADR from tweet:  tendon rupture

Matched ADR term in corpus: tendon rupture 
Matched ADR code 10043248 (Score: 1.0000)

Expected ADR term :  tendon rupture 
Expected ADR code :  10043248

Result: Correct




Extracted ADR from tweet:  withdrawal

Matched ADR term in corpus: withdrawn 
Matched ADR code 10048013 (Score: 0.9536)

Expected ADR term :  withdrawal syndrome 
Expected ADR code :  10048010

Result: Wrong




Extracted ADR from tweet:  emotional mess

Matched ADR term in corpus: instability emotional 
Matched ADR code 10022454 (Score: 0.9321)

Expected ADR term :  emotional distress 
Expected ADR code :  10049119

Result: Wrong




Extracted ADR from tweet:  not remember

Matched ADR term in corpus: unable to walk 
Matched ADR code 10049278 (Score: 0.8874)

Expected ADR term :  memory loss 
Expected ADR code :  10027176

Result: Wrong




Extracted ADR from tweet:  memory lapse

Matched ADR term in corpus: memory los